In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
from api.utils import parse_offset, today


In [ ]:
from api.gui.radar2 import EditableGrid
from api.gui.collectors import DataCollector
from api.gui.params import curves

In [ ]:
curves_df = pd.DataFrame(curves).set_index('ccy').loc[['EUR', 'USD', 'GBP']]

In [ ]:
self = EditableGrid()

In [ ]:
from api import giobhutils as gio
from api.ezutils import SQL

In [ ]:
def get_var_query( 
    asof=None,
    book='MM',
    offset='2y'
):
    dt_fmt = '%Y%m%d'
    asof = pd.to_datetime(asof) if asof else today() - BDay()
    query = f"SELECT " \
        f"p.fund, " \
        f"p.book, " \
        f"p.strategy, " \
        f"p.minnb, " \
        f"p.positionName, " \
        f"p.riskCategory, " \
        f"p.currency, " \
        f"p.instrument, " \
        f"p.underlying_name, " \
        f"p.bhType, " \
        f"p.isAO, " \
        f"s.returnDate, " \
        f"s.returnValue " \
        f"FROM RiskManagement..vw_pos_openPositionMarketData p " \
        f"JOIN RiskVaR..vw_SeriesData s ON p.cobDate = s.cobDate and p.minbhid = s.minbhid " \
        f"WHERE 1=1 " \
        f"AND p.cobDate = '{asof:{dt_fmt}}' " \
        f"AND p.fund = 'BAL' " \
        f"AND p.book = '{book}' " \
        f"AND s.riskType = 'Total' " \
        f"AND s.returnHorizon = '1D' " \
        f"AND s.analysisHorizon = '1D' " \
        f"AND s.decayFactor = 1 " \
        f"AND s.exclude = 0 "
    
    if offset:
        offset = asof - parse_offset(offset)
        query = f"{query} AND s.returnDate >= {offset:{dt_fmt}}"
    return query

In [ ]:
get_var_query(offset='2y')

In [ ]:
res = gio.bhDbReadSql(
    '1', 
    sqlserver=SQL.RiskVar.server,
    sqldatabase=SQL.RiskVar.db,
    sqlquery=get_var_query(offset='2y')
)

In [ ]:
res.py

In [ ]:
self

In [ ]:
dc = DataCollector(
    curves=curves_df.melt()['value'].drop_duplicates().dropna().to_list(),
    collectors={
        'eg': self
    },
)

In [ ]:
dc.get_data()

In [ ]:
self = Radar(curves=curves_df.melt()['value'].to_list())
self

In [ ]:
dc.db_curves